In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

import xgboost as xgb

In [ ]:
# Read from South River historcal data
df = pd.read_parquet(r'C:\Users\JohnSellers\Downloads\South_River_Mar_Aug.parquet')

# Save to csv
df.to_csv(r'C:\Users\JohnSellers\Downloads\South_River_Mar_Aug.csv')

In [ ]:
## Set time column as index
df['time'] = pd.to_datetime(df['time']).dt.tz_convert('US/Eastern')

## Build clean_df
bms01_df = pd.DataFrame()

bms01_df['time'] = df['time']

## Site SOC is the average SOC of all ESS
# clean_df['Site_SOC'] = soc_df.mean(axis = 1)
bms01_df['ESS001_BMS_soc'] = df['ESS001_BMS01_soc']

# Determing BMS # to dechipher between BMS01 and BMS02
bms01_df['BMS'] = 'BMS01'


## Add meters data
bms01_df['Site_kwh_Delivered'] = df['sel_735_kwh_delivered']
bms01_df['Site_kwh_Received'] = df['sel_735_kwh_received']
bms01_df['Site_Active_Power'] = df['sel_735_active_power']

## Calculate SOC difference to divide into individual charging/discharging events
bms01_df['SOC_Diff'] = bms01_df.ESS001_BMS_soc.diff()
bms01_df['Delivered_Diff'] = bms01_df.Site_kwh_Delivered.diff()
bms01_df['Received_Diff'] = bms01_df.Site_kwh_Received.diff()

# # Viewing the dataframe to ensure it is correct
# bms01_df

In [ ]:
## If diff is greater than 0, it's charging
## If diff is less than 0, it is discharging 
discharging_timestamp = False
charging_timestamp = False
# nameplate_capacity = 21600
# energy_per_percent = nameplate_capacity / 100

recal_df = pd.DataFrame(columns = ['Start Time', 'Starting SOC','Starting Energy', 'End Time', 'Ending SOC', 'Ending Energy', 'Type', 'SOC Delta', 'Energy Delta', 'Energy per Percent SOC'])

for index, row in bms01_df.iterrows():

    # a positive difference indicates a possible charging period
    if row['SOC_Diff'] > 0:
        # if we do not already have a possible starting charging time, grab the values
        if not charging_timestamp and not (row['ESS001_BMS_soc'] > 80):
            charging_timestamp = row['time']
            starting_soc = row['ESS001_BMS_soc']
            starting_energy = row['Site_kwh_Received']
        # if we have a possible starting discharge time, clear it
        # if it was a true discharge, soc would not increase
        elif discharging_timestamp:
            discharging_timestamp = False

    # a negative difference indicates a possible discharge period
    elif row['SOC_Diff'] < 0 and not (row['ESS001_BMS_soc'] < 15):
        # if we do not already have a possible starting discharging time, grab the values
        if not discharging_timestamp:
            discharging_timestamp = row['time']
            starting_soc = row['ESS001_BMS_soc']
            starting_energy = row['Site_kwh_Delivered']
        # if we have a possible starting charging time, clear it
        # if it was a true charge, soc would not decrease
        elif charging_timestamp:
            charging_timestamp = False

    # when the difference is 0, we check to see if a charge or discharge has finished
    elif row['SOC_Diff'] == 0:
        # if we have a possible discharge, check to see if soc is below 15
        if discharging_timestamp:
            if row['ESS001_BMS_soc'] < 15:
                # we have found a discharging event ! yay !
                ending_soc = row['ESS001_BMS_soc']
                ending_energy = row['Site_kwh_Delivered']
                soc_delta = starting_soc - ending_soc
                energy_delta = ending_energy - starting_energy
                energy_per_soc = energy_delta / soc_delta
                recal_df.loc[len(recal_df)] = [discharging_timestamp, starting_soc, starting_energy, row['time'], ending_soc, ending_energy, 'Discharge', soc_delta, energy_delta, energy_per_soc]
                discharging_timestamp = False
            else:
                discharging_timestamp = False
        # if we have a possible charge, check to see if soc is above 95
        elif charging_timestamp:
            if row['ESS001_BMS_soc'] > 95:
                # we have found a charging event ! yay !
                ending_soc = row['ESS001_BMS_soc']
                ending_energy = row['Site_kwh_Received']
                soc_delta = ending_soc - starting_soc
                energy_delta = ending_energy - starting_energy
                energy_per_soc = energy_delta / soc_delta
                recal_df.loc[len(recal_df)] = [charging_timestamp, starting_soc, starting_energy, row['time'], ending_soc, ending_energy, 'Charge', soc_delta, energy_delta, energy_per_soc]
                charging_timestamp = False
            else:
                charging_timestamp = False

# # Viewing new dataframe to see if it looks correct
# recal_df

In [ ]:
# Selecting all observations that fall betwwen recal_df['Start Time'] and recal_df['End Time'] in bms01_df and creating 
# a new dataframe with these observations. This dataframe will be used for training, validating and testing the model
df = pd.DataFrame()
for i in range(len(recal_df)):
    df = pd.concat([df, bms01_df[(bms01_df['time'] >= recal_df['Start Time'][i]) & (bms01_df['time'] <= recal_df['End Time'][i])]])

obs_df = df.reset_index(drop=True)
# obs_df

In [ ]:
# Determind when the observation is charging, discharging, or idle so that we can break down the data into charge, discharge, and idle dataframes
obs_df['Type'] = np.where(obs_df['ESS001_BMS_soc'].shift(-1) > obs_df['ESS001_BMS_soc'], 'Charge', 'Discharge')
# obs_df

In [ ]:
# Breaking down the obs_df into charge and discharge cycles and saving them as separate charge_df and discharge_df
charge_df = obs_df[obs_df['Type'] == 'Charge']
discharge_df = obs_df[obs_df['Type'] == 'Discharge']

# Checking the unique values in the 'Type' column to ensure that the above operation was successful
# charge_df

In [ ]:
# discharge_df

In [ ]:
# Creating a function to create a basic linear regression model
# This function will be used to create a linear regression model trained on the training data
# The model will then be used to predict the target variable for the validation and testing data
# The function will also return the mean squared error and r-squared values for the validation and testing data

def linear_regression_model(X, y, train_size, test_size, seed):
    # Splitting the whole dataset into training and remainder sets
    # The remainder with be split into validation and test sets
    X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=train_size, random_state=seed)

    # Splitting the remainder set into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=test_size, random_state=seed)

    # # Printing the shapes of the training, validation, and test sets to ensure that the proportions are correct
    # print(X_train.shape, y_train.shape)
    # print(X_val.shape, y_val.shape)
    # print(X_test.shape, y_test.shape)

    # Creating a linear regression object
    lin_reg = LinearRegression()
    
    # Training the linear regression model on the training data
    pred = lin_reg.fit(X_train, y_train)
    
    # Predicting the target variable for all three datasets
    y_train_pred = pred.predict(X_train)
    y_val_pred = pred.predict(X_val)
    y_test_pred = pred.predict(X_test)

    # Storing predicted and actual values in a dataframe to be viewd later and compared
    predicted_train_df = pd.DataFrame({"predicted":y_train_pred, "actual": y_train}).reset_index(drop=True)
    predicted_val_df = pd.DataFrame({"predicted":y_val_pred, "actual": y_val}).reset_index(drop=True)
    predicted_test_df = pd.DataFrame({"predicted":y_test_pred, "actual": y_test}).reset_index(drop=True)

    # Viewing the predicted values vs the actual values of the linear regression model to see how well it fits the data
    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_train_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Training Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()

    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_val_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Validation Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()

    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_test_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Test Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()
    
    # Calculating the mean squared error for all three datasets
    train_mse = mean_squared_error(y_train, y_train_pred)
    val_mse = mean_squared_error(y_val, y_val_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Calculating the r-squared value for all three datasets
    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    return pd.DataFrame({
        'Train MSE':train_mse, 
        'Train R2':train_r2, 
        'Val MSE':val_mse, 
        'Val R2':val_r2, 
        'Test MSE':test_mse, 
        'Test R2':test_r2},
        index=[0])

In [ ]:
# Creating a function to create a basic random forest model
# This function will be used to create a random forest regression model trained on the training data
# The model will then be used to predict the target variable for the validation and testing data
# The function will also return the mean squared error and r-squared values for the validation and testing data

def random_forest_model(X, y, train_size, test_size, seed, n_estimators, max_depth, max_leaf_nodes):
    # Splitting the whole dataset into training and remainder sets
    # The remainder with be split into validation and test sets
    X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=train_size, random_state=seed)

    # Splitting the remainder set into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=test_size, random_state=seed)

    # Printing the shapes of the training, validation, and test sets to ensure that the proportions are correct
    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)
    print(X_test.shape, y_test.shape)

    # Creating a linear regression object
    rf_reg = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_leaf_nodes=max_leaf_nodes)

    # Training the linear regression model on the training data
    pred = rf_reg.fit(X_train, y_train)
    
    # Predicting the target variable for all three datasets
    y_train_pred = pred.predict(X_train)
    y_val_pred = pred.predict(X_val)
    y_test_pred = pred.predict(X_test)

    # Storing predicted and actual values in a dataframe to be viewd later and compared
    predicted_train_df = pd.DataFrame({"predicted":y_train_pred, "actual": y_train}).reset_index(drop=True)
    predicted_val_df = pd.DataFrame({"predicted":y_val_pred, "actual": y_val}).reset_index(drop=True)
    predicted_test_df = pd.DataFrame({"predicted":y_test_pred, "actual": y_test}).reset_index(drop=True)
    
    
    # Viewing the predicted values vs the actual values of the linear regression model to see how well it fits the data
    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_train_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Training Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()

    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_val_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Validation Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()

    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_test_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Test Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()
    
    # Calculating the mean squared error for all three datasets
    train_mse = mean_squared_error(y_train, y_train_pred)
    val_mse = mean_squared_error(y_val, y_val_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Calculating the r-squared value for all three datasets
    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Returning the mean squared error and r-squared values for all three datasets
    return pd.DataFrame({
        'Train MSE':train_mse, 
        'Train R2':train_r2, 
        'Val MSE':val_mse, 
        'Val R2':val_r2, 
        'Test MSE':test_mse, 
        'Test R2':test_r2},
        index=[0])

In [ ]:
# Creating a function to create a basic random forest model
# This function will be used to create a random forest regression model trained on the training data
# The model will then be used to predict the target variable for the validation and testing data
# The function will also return the mean squared error and r-squared values for the validation and testing data

def xgboost_reg_model(X, y, train_size, test_size, seed, n_estimators, learning_rate, max_depth): #, n_estimators, max_depth, max_leaves
    # Splitting the whole dataset into training and remainder sets
    # The remainder with be split into validation and test sets
    X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=train_size, random_state=seed)

    # Splitting the remainder set into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=test_size, random_state=seed)

    # Printing the shapes of the training, validation, and test sets to ensure that the proportions are correct
    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)
    print(X_test.shape, y_test.shape)


    # Creating a linear regression object
    xgb_reg = xgb.XGBRegressor(objective='reg:linear', n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth) #n_estimators, max_depth, max_leaves

    # Training the linear regression model on the training data
    pred = xgb_reg.fit(X_train, y_train)
    
    # Predicting the target variable for all three datasets
    y_train_pred = pred.predict(X_train)
    y_val_pred = pred.predict(X_val)
    y_test_pred = pred.predict(X_test)

    # Storing predicted and actual values in a dataframe to be viewd later and compared
    predicted_train_df = pd.DataFrame({"predicted":y_train_pred, "actual": y_train}).reset_index(drop=True)
    predicted_val_df = pd.DataFrame({"predicted":y_val_pred, "actual": y_val}).reset_index(drop=True)
    predicted_test_df = pd.DataFrame({"predicted":y_test_pred, "actual": y_test}).reset_index(drop=True)
    
    
    # Viewing the predicted values vs the actual values of the linear regression model to see how well it fits the data
    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_train_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Training Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()

    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_val_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Validation Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()

    with pd.option_context('plotting.backend', "plotly"):
        fig = predicted_test_df.head(1000).plot().update_layout(title = "Predicted vs Actual Values for Test Data", xaxis_title="Features", yaxis_title="BMS SOC")
        fig.show()
    
    # Calculating the mean squared error for all three datasets
    train_mse = mean_squared_error(y_train, y_train_pred)
    val_mse = mean_squared_error(y_val, y_val_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # Calculating the r-squared value for all three datasets
    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    # Returning the mean squared error and r-squared values for all three datasets
    return pd.DataFrame({
        'Train MSE':train_mse, 
        'Train R2':train_r2, 
        'Val MSE':val_mse, 
        'Val R2':val_r2, 
        'Test MSE':test_mse, 
        'Test R2':test_r2},
        index=[0])

# Charge/Discharge Observation

### Linear Regression

In [ ]:
cd_lr = linear_regression_model(
    obs_df[['Site_kwh_Delivered', 'Site_kwh_Received', 'Received_Diff']], 
    obs_df['ESS001_BMS_soc'], 
    train_size=0.75, 
    test_size=0.5,
    seed=0
)

### Random Forest Regressor

In [ ]:
cd_rf = random_forest_model(
    obs_df[['Site_kwh_Delivered','Site_kwh_Received', 'Received_Diff']], 
    obs_df['ESS001_BMS_soc'], 
    train_size=0.80, 
    test_size=0.5, 
    seed=0, 
    n_estimators=200, 
    max_depth=15,
    max_leaf_nodes=15
)

### XGBoost Regressor

In [ ]:
cd_xgb = xgboost_reg_model(
    obs_df[['Site_kwh_Delivered','Site_kwh_Received', 'Received_Diff']], 
    obs_df['ESS001_BMS_soc'], 
    train_size=0.80, 
    test_size=0.5, 
    seed=0,
    n_estimators=200,
    learning_rate=.1,
    max_depth=5,
    )

# Charging Cycle

### Linear Regression

In [ ]:
c_lr = linear_regression_model(
    charge_df[['Site_kwh_Delivered', 'Site_kwh_Received', 'Received_Diff']], 
    charge_df['ESS001_BMS_soc'], 
    train_size=0.75, 
    test_size=0.5, 
    seed=0)

### Random Forest Regressor

In [ ]:
c_rf = random_forest_model(
    charge_df[['Site_kwh_Delivered','Site_kwh_Received', 'Received_Diff']], 
    charge_df['ESS001_BMS_soc'], 
    train_size=0.80, 
    test_size=0.5, 
    seed=0, 
    n_estimators=200, 
    max_depth=15,
    max_leaf_nodes=15
)

### XGBoost Regressor

In [ ]:
c_xgb = xgboost_reg_model(
    charge_df[['Site_kwh_Delivered','Site_kwh_Received', 'Received_Diff']], 
    charge_df['ESS001_BMS_soc'], 
    train_size=0.80, 
    test_size=0.5, 
    seed=0,
    n_estimators=200,
    learning_rate=.1,
    max_depth=5,
    )

# Discharging Cycle

### Linear Regression

In [ ]:
d_lr = linear_regression_model(
    discharge_df[['Site_kwh_Delivered', 'Site_kwh_Received', 'Received_Diff']], 
    discharge_df['ESS001_BMS_soc'], 
    train_size=0.75, 
    test_size=0.5, 
    seed=0
)

### Random Forest Regressor

In [ ]:
d_rf = random_forest_model(
    discharge_df[['Site_kwh_Delivered', 'Site_kwh_Received', 'Received_Diff']], 
    discharge_df['ESS001_BMS_soc'], 
    train_size=0.75, 
    test_size=0.5, 
    seed=0, 
    n_estimators=200, 
    max_depth=15, 
    max_leaf_nodes=15)

### XGBoost Regressor

In [ ]:
d_xgb = xgboost_reg_model(
    discharge_df[['Site_kwh_Delivered','Site_kwh_Received', 'Received_Diff']], 
    discharge_df['ESS001_BMS_soc'], 
    train_size=0.80, 
    test_size=0.5, 
    seed=0,
    n_estimators=200,
    learning_rate=.1,
    max_depth=5
    )

In [ ]:
metrics_df = pd.concat([cd_lr, cd_rf, cd_xgb, c_lr, c_rf, c_xgb, d_lr, d_rf, d_xgb], axis=0).reset_index(drop=True)
metrics_df.index = ['CD Linear Regression', 'CD Random Forest', 'CD XGBoost', 'C Linear Regression', 'C Random Forest', 'C XGBoost', 'D Linear Regression', 'D Random Forest', 'D XGBoost']
metrics_df